In [16]:
import numpy as np
import pandas as pd
import sympy as sp
from fractions import Fraction
import itertools
import timeit

### Funciones previas

In [17]:
def permutation(lst):
    
    if len(lst) == 0:
        return []
    if len(lst) == 1:
        return [lst]
    
    l = []

    for i in range(len(lst)):
        m = lst[i]
        remLst = lst[:i] + lst[i+1:]

        for p in permutation(remLst):
            l.append([m] + p)
        
    return l

# Paridad de las permutaciones (solo funciona con el orden de la funcion anterior)

def par(lst):
    
    l = []
    
    for i in range(len(lst)):
        if i % 2 == 0:
            l.append(1)
            
        else:
            l.append(-1)
    return np.cumprod(np.array(l))

def combinations(n,k):
    
    u = list(range(0,n))
    
    c = [list(v) for v in itertools.product(u, repeat=k) if len([i for i in v if v.count(i)>1])==0]
    
    return c 
 
def kCombinations(output, temp, index, n, i, K):
    if index == K:
        output.append(temp.copy())
        return
    
    if i >=n:
        return
    
    temp[index] = i #+ 1
    kCombinations(output, temp, index + 1, n, i + 1, K)
    kCombinations(output, temp, index, n, i + 1, K)
    
def findKCombination(K,n):
    
    output = []
    temp = [0]*K
    kCombinations(output, temp, 0, n, 0, K)
    return output

### Delta Generalizada

In [18]:
def Delta(upI,lwI):
    
    per = permutation(lwI)
    pari = par(per)
    
    # Verifica si hay indices repetidos
    
    if len([i for i in upI if upI.count(i)>1]) > 0:
        return 0
    if len([i for i in lwI if lwI.count(i)>1]) > 0:
        return 0  
    
    # Busca si los indices superiores son una permutacion (par o impar) del los inferiores
    
    if upI in per:
        k = per.index(upI)
        return sp.Rational(pari[k],np.math.factorial(len(upI)))
    else:
        return 0

### Check de traza

Deberia ser 

$\Delta^{\mu_1 ... \mu_k}_{\nu_1 ... \nu_k} = \frac{1}{k!}\frac{n!}{(n-k)!}$

In [43]:
def TraDelta(n,k):
    u = 0
    for i in combinations(n,k):
        u = u + Delta(i,i)
    return u  

def checkDelta(n,k):
    
    #a = np.math.factorial(n)/np.math.factorial(n-k)
    #b = np.math.factorial(k)
    #s = (1/b)*a
    
    s = sp.Rational(1,np.math.factorial(k))*sp.Rational(np.math.factorial(n),np.math.factorial(n-k))
    
    if TraDelta(n,k) == s:
        return True
    else: 
        return False

In [39]:
%timeit -r 4 -n 10000 checkDelta(4,1)

42 µs ± 442 ns per loop (mean ± std. dev. of 4 runs, 10,000 loops each)


In [40]:
%timeit -r 4 -n 10000 checkDelta2(4,1)

18.6 µs ± 397 ns per loop (mean ± std. dev. of 4 runs, 10,000 loops each)


In [27]:
checkDelta(4,2)

False

In [28]:
checkDelta(4,3)

False

In [29]:
checkDelta(5,2)

False

In [30]:
checkDelta(5,3)

False

In [31]:
checkDelta(5,4)

False

### Representacion fundamental de SU(n)

In [44]:
def Li(n,i,j,a,b):
    
    s = Delta([a],[j])*Delta([i],[b])-Delta([i],[j])*Delta([a],[b])*sp.Rational(1,n)
    
    return s

def LiM(n,i,j):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(0,n):
            u[a][b]=Li(n,i,j,a,b)
    return u   

def L2(n,i,j):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(a,n):
            u[a][b] = (sp.Rational(1,2))*(Li(n,i,j,a,b)+Li(n,j,i,a,b))
            u[b][a] = u[a][b]
            
    return u          

def M2(n,i,k):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(a,n):
            u[a][b] = (sp.Rational(1,2))*(1j)*(Li(n,i,k,a,b)-Li(n,k,i,a,b))
            u[b][a] = -u[a][b]
            
    return u

def D(n,k):
    
    u = sp.sqrt(2*(k+1)*(k+2))
    
    s = np.zeros((n,n),dtype=object)
    
    for i in range(0,k+1):
        s = s + LiM(n,i,i)
        
    s = s - (k+1)*LiM(n,k+1,k+1)
    
    return s*(1/u)

In [45]:
def GenLamda3(n):
    
    p = []
    b = []

    for j in range(1,n):
        for i in range(0,j):
           # if i >= j:
           #     continue    

            p.append(2*L2(n,i,j))
            b.append('s')
            p.append(2*M2(n,i,j))
            b.append('as')
        p.append(2*D(n,j-1))
        b.append('d')
    return p,b

### Traza de los generadores $\lambda_I$

In [7]:
def Traza(A,B):
    
    u = np.trace(np.matmul(A,B,dtype=object))
    
    return u

In [8]:
def checkTL(Lamdas):
    
    u = (Lamdas[0].shape)[0]
    s = []
    
    for i,j in itertools.product(range(0,u*u-1), repeat=2):
        
        if Traza(Lamdas[i],Lamdas[j]) == 2*Delta([i],[j]):
            s.append(True)
        else:
            s.append(False)
    
    
    if all(s) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
    
    

In [15]:
checkTL(GenLamda3(2)[0])

'Todas las relaciones se cumplen'

In [16]:
checkTL(GenLamda3(3)[0])

'Todas las relaciones se cumplen'

In [17]:
checkTL(GenLamda3(4)[0])

'Todas las relaciones se cumplen'

In [18]:
checkTL(GenLamda3(5)[0])

'Todas las relaciones se cumplen'

In [19]:
checkTL(GenLamda3(6)[0])

'Todas las relaciones se cumplen'

### Calculo de los $t_I$

$(t_I)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = (-1)^{k-1}(k-1)!\delta_{l_1}^{[\mu_1}\Delta^{\mu_2 \ldots \mu_k]\rho_1}_{\nu_1 \ldots \nu_k}(\lambda_I)_{\rho_1}^{l_1}$

Estos generadores satisfacen la condicion:

$Tr(t_It_J) = \frac{2(k-1)!(n-2)!}{k^2(n-k-1)!}\delta_{IJ}$

$[t_I,t_J]^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = 2i\frac{(k-1)!}{k}f_{IJ}^K (t_K)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} $

Resulta mas comodo definir todo como


$(t_I)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = (-1)^{k-1}\frac{2}{k}\delta_{l_1}^{[\mu_1}\Delta^{\mu_2 \ldots \mu_k]\rho_1}_{\nu_1 \ldots \nu_k}(\lambda_I)_{\rho_1}^{l_1}$


$Tr(t_It_J) = \frac{(n-2)!}{2(k-1)!(n-k-1)!}\delta_{IJ}$

$[t_I,t_J]^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = if_{IJ}^K (t_K)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} $


In [9]:
def Pro(UpI,LwI):
    
    s = 0
    
    perU = permutation(UpI[:len(UpI)-1])
    pari = par(perU)
    
    if len([i for i in UpI[:len(UpI)-1] if UpI[:len(UpI)-1].count(i)>1]) > 0:
        return 0
    else:
        for i in perU:
            k = perU.index(i)
            s = s + Delta([i[0]],[LwI[0]])*Delta(i[1:]+[UpI[-1]],LwI[1:])*pari[k]*Fraction(1,np.math.factorial(len(UpI)-1))
    return s 

In [10]:
def t3(n,matrix,upI,lwI):
    
    i = 0
    
    sig = ((-1)**(len(upI)-1))*sp.Rational(len(upI),2)
    
    s,t = np.where(matrix !=0 )
    
    
    for p,m in zip(s,t):
        UpI = upI + [p]
        LpI = [m] + lwI
        i = i + Pro(UpI,LpI)*(matrix)[p][m]
    
    return i*sig

### Verifiquemos la traza

In [11]:
def Tract(matrix,k):
    
    u = (matrix[0].shape)[0]
    
    p = combinations(u,k)
    
    rs = sp.Rational(np.math.factorial(u-2),2*np.math.factorial(k-1)*np.math.factorial(u-k-1))
    
    t = []
    
    for i,j in itertools.product(range(0,u*u-1), repeat=2):
        
        l = []
        
        for upI in p:
            for lwI in p:
                l.append(sp.nsimplify(t3(u,matrix[i],upI,lwI)*t3(u,matrix[j],lwI,upI)))
                
  #      print(sp.nsimplify(sum(l)),sp.nsimplify(rs*Delta([i],[j])))
        
        if sp.nsimplify(sum(l)) == sp.nsimplify(rs*Delta([i],[j])):
            t.append(True)
        else:
            t.append(False)
            
    if all(t) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'            

Esto tarda mucho

In [23]:
Tract(GenLamda3(3)[0],2)

'Todas las relaciones se cumplen'

In [24]:
Tract(GenLamda3(4)[0],2)

KeyboardInterrupt: 

In [ ]:
Tract(GenLamda3(4)[0],3)

In [ ]:
Tract(GenLamda3(5)[0],2)

### Veamos el bracket

In [12]:
def brat(n,A,B,upI,lwI):
    
    k = len(upI)
    u = 0
    
    p = combinations(n,k)
        
    for s in p:
        u = u + (t3(n,A,s,lwI)*t3(n,B,upI,s) - t3(n,B,s,lwI)*t3(n,A,upI,s))
        
    return u    
    

## Representacion Matricial de los generadores

In [13]:
def tM(n,Ma,st,k):
    
    values = findKCombination(k,n)
    dim = len(values)
    keys = [i for i in range(0,dim)]
    
    dic = dict(zip(keys,values))
    
    matrix = np.zeros((dim,dim),dtype=object)
    
    if st == 's':
        
        for i in range(0,dim):
            for j in range(i,dim):
                matrix[i][j] = np.math.factorial(k)*t3(n,Ma,dic[j],dic[i])
                matrix[j][i] = matrix[i][j]
                
    elif st == 'as':
        
        for i in range(0,dim):
            for j in range(i,dim):
                matrix[i][j] = np.math.factorial(k)*t3(n,Ma,dic[j],dic[i])
                matrix[j][i] = -matrix[i][j]
                
    elif st == 'd':
        
        for i in range(0,dim):
            matrix[i][i] = np.math.factorial(k)*t3(n,Ma,dic[i],dic[i])       
            
    return matrix

In [14]:
def tMf(n,k):
    
    ##############################################################
    ############ Todas las matrices y su simetria ################
    ##############################################################
    
    Lamda, Sim = GenLamda3(n)
    
    ###############################################################
    ######## Representacion Matricial de los T ####################
    ###############################################################
    
    a = []
    
    for u in range(0,n*n-1):
        
        a.append(tM(n,Lamda[u],Sim[u],k))
    
    return Lamda,a

In [15]:
def bra(A,B):
    
    return np.matmul(A,B,dtype=object)-np.matmul(B,A,dtype=object)

### Anticomutador


def Abra(A,B):
    
    return np.matmul(A,B,dtype=object)+np.matmul(B,A,dtype=object)



def f3(LamdL,n):
       
    u = range(0,n*n-1)
    
   # values = [list(v) for v in itertools.product(u, repeat=3)]
    di = {}
    
    for i,j,k in itertools.product(u, repeat=3):
        
        m = sp.nsimplify(-(1j)*sp.Rational(1,4)*np.trace(np.matmul(LamdL[i],bra(LamdL[j],LamdL[k]),dtype=object)))
        di[(i,j,k)] = m
        
    return di    



## Generadores $T_I$ 

En esta parte se calculan todos los $T_I$ en su representacion matricial. Por simplicidad aqui calculare todos los generadores para $n=3$,$n=4$,$n=5$. Para todos los valores de $2 \leq k < n$.

### n=3

In [16]:
# k=2

Lamda_3, T32 = tMf(3,2)

In [17]:
F_3 = f3(Lamda_3,3)

### n=4

In [18]:
# k = 2

Lamda_4, T42 = tMf(4,2) 

# k =3
T43 = tMf(4,3)[1]

In [19]:
F_4 = f3(Lamda_4,4)

### n = 5

In [20]:
# k = 2

Lamda_5, T52 = tMf(5,2)

# k = 3

T53 = tMf(5,3)[1]

# k = 4

T54 = tMf(5,4)[1]

In [21]:
F_5 = f3(Lamda_5,5)

### Pruebas de traza y algebra

In [22]:
## Comutador



## Constantes de estructura de SU(n)

def f2(LamdL,I,J,K):
    
    if I == J or J == K:
        return 0
    else:
        return  sp.nsimplify(-(1j)*sp.Rational(1,4)*np.trace(np.matmul(LamdL[I],bra(LamdL[J],LamdL[K]),dtype=object)))
    


### Algebra

In [23]:
def rhs(n,I,J,Mat,Lamda,st):
    
    a = np.zeros((len(Mat[0]),len(Mat[0])),dtype=object)
    
    for i in range(0,n*n-1):
        if st == 'l':
            p = 2j
        elif st == 't':
            p = 1j
        elif st == 'T':
            p = 1
        u = f2(Lamda,I,J,i)*p*Mat[i]
        u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
        a = a + u
        
    return a

def rhs2(n,I,J,Mat,estc,st,k):
    
    a = np.zeros((len(Mat[0]),len(Mat[0])),dtype=object)
    
    for i in range(0,n*n-1):
        if st == 'l':
            p = 2j
        elif st == 't':
            p = 1j
        elif st == 'T':
            p = 1
        u = estc[(I,J,i)]*p*Mat[i]
        u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
        a = a + u
        
    return a
    

In [24]:
def alg(n,I,J,Mat,Lamda,st,k):
    
    a = rhs(n,I,J,Mat,Lamda,st)
        
    b = bra(Mat[I],Mat[J])
    
    ## Verifiquemos que todo sea del mismo tipo
    
    a[a!=0] = np.array([sp.nsimplify(i) for i in a[a!=0]],dtype=object)
    b[b!=0] = np.array([sp.nsimplify(i) for i in b[b!=0]],dtype=object)
    
    if (b == a).all():
        return True
    else:
        return False
    
def alg2(n,I,J,Mat,estc,st,k):
    
    a = rhs2(n,I,J,Mat,estc,st,k)
        
    b = bra(Mat[I],Mat[J])
    
    ## Verifiquemos que todo sea del mismo tipo
    
    a[a!=0] = np.array([sp.nsimplify(i) for i in a[a!=0]],dtype=object)
    b[b!=0] = np.array([sp.nsimplify(i) for i in b[b!=0]],dtype=object)
    
    if (b == a).all():
        return True
    else:
        return False    

### Vamos ahora con las pruebas para cada caso

In [25]:
def Algtest(n,Mat,Lamda,st,k):
    
    a = []
    
    for i,j in combinations(n*n-1,2):
        a.append(alg(n,i,j,Mat,Lamda,st))
        
        # Para saber en que indices no se cumple
        
        # if alg(n,i,j,Mat,st) == False:
            
        #     print(i,j)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
    
def Algtest2(n,Mat,estc,st,k):
    
    a = []
    
    for i,j in combinations(n*n-1,2):
        a.append(alg2(n,i,j,Mat,estc,st,k))
        
        # Para saber en que indices no se cumple
        
        #if alg2(n,i,j,Mat,estc,st) == False:
            
        #    print(i,j)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'    

### Verificando las relaciones de conmutacion de los $\lambda_I$

In [39]:
Algtest2(3,Lamda_3,F_3,'l',1)

'Todas las relaciones se cumplen'

In [40]:
Algtest2(4,Lamda_4,F_4,'l',1)

'Todas las relaciones se cumplen'

In [41]:
Algtest2(5,Lamda_5,F_5,'l',1)

'Todas las relaciones se cumplen'

### Verificando las relaciones de commutacion de los $t_I$

In [42]:
Algtest2(3,T32,F_3,'t',2)

'Todas las relaciones se cumplen'

In [43]:
Algtest2(4,T42,F_4,'t',2)

'Todas las relaciones se cumplen'

In [44]:
Algtest2(4,T43,F_4,'t',3)

'Todas las relaciones se cumplen'

In [45]:
Algtest2(5,T52,F_5,'t',2)

'Todas las relaciones se cumplen'

In [46]:
Algtest2(5,T53,F_5,'t',3)

'Todas las relaciones se cumplen'

In [47]:
Algtest2(5,T54,F_5,'t',4)

'Todas las relaciones se cumplen'

### Verificando la traza

In [26]:
def checkTTmat(tmat,n,k):
    
    s = []
    
    rs = sp.Rational(np.math.factorial(n-2),2*np.math.factorial(k-1)*np.math.factorial(n-k-1))
    
    for i,j in itertools.product(range(0,n*n-1), repeat=2):
        
        if Traza(tmat[i],tmat[j]) == rs*Delta([i],[j]):
            s.append(True)
        else:
            print(Traza(tmat[i],tmat[j]),rs*Delta([i],[j]))
            s.append(False)
    
    
    if all(s) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'

In [49]:
checkTTmat(T32,3,2)

'Todas las relaciones se cumplen'

In [50]:
checkTTmat(T42,4,2)

'Todas las relaciones se cumplen'

In [51]:
checkTTmat(T43,4,3)

'Todas las relaciones se cumplen'

In [52]:
checkTTmat(T52,5,2)

'Todas las relaciones se cumplen'

In [53]:
checkTTmat(T53,5,3)

'Todas las relaciones se cumplen'

In [54]:
checkTTmat(T54,5,4)

'Todas las relaciones se cumplen'

### Vamos ahora a calcular los $T_I$

En base a los cambios anteriores yo colocaria 

$(T_I)^{A}_{B} = i(t^t_I)^{A}_{B}$

In [27]:
def Ts(tm):
    
    s = tm[0].shape
    a = []
    
    for k in tm:
        
        u = np.array(np.zeros((s[0]+4,s[0]+4),dtype=object))
        u[4:,4:] = (1j)*np.transpose(k)
        a.append(u)
        
    return a   

In [28]:
TF32 = Ts(T32)
TF42 = Ts(T42)
TF43 = Ts(T43)
TF52 = Ts(T52)
TF53 = Ts(T53)
TF54 = Ts(T54)

In [113]:
Algtest2(3,TF32,F_3,'T',2)

'Todas las relaciones se cumplen'

In [114]:
Algtest2(4,TF42,F_4,'T',2)

'Todas las relaciones se cumplen'

In [115]:
Algtest2(4,TF43,F_4,'T',3)

'Todas las relaciones se cumplen'

In [116]:
Algtest2(5,TF52,F_5,'T',2)

'Todas las relaciones se cumplen'

In [117]:
Algtest2(5,TF53,F_5,'T',3)

'Todas las relaciones se cumplen'

In [118]:
Algtest2(5,TF54,F_5,'T',4)

'Todas las relaciones se cumplen'

## Verifiquemos la propiedad $T_I^\dagger = - T_I$

In [29]:
def anticheck(Matrix):
    
    a = []
    
    for i in Matrix:
        
        u = np.conjugate(np.transpose(i))
        
        if (u == -i).all():
            a.append(True)
        else:
            a.append(False)
            
    
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'            

In [64]:
anticheck(TF32)

'Todas las relaciones se cumplen'

In [65]:
anticheck(TF42)

'Todas las relaciones se cumplen'

In [66]:
anticheck(TF43)

'Todas las relaciones se cumplen'

In [67]:
anticheck(TF52)

'Todas las relaciones se cumplen'

In [68]:
anticheck(TF53)

'Todas las relaciones se cumplen'

In [69]:
anticheck(TF54)

'Todas las relaciones se cumplen'

In [30]:
def Gamma(n,k):
    
    d = TraDelta(n,k)
    
    u = np.array(np.zeros((d+4,d+4),dtype=object))

    for i in range(0,4): u[i][i] = 1
    
    for i in range(4,d+4): u[i][i] = -1#sp.Rational(d,np.math.factorial(k))
    
    return u
    

In [31]:
def superTr(n,k,mat):
    
    s = Gamma(n,k)
    
    l = []
    
    rs = sp.Rational(np.math.factorial(n-2),2*np.math.factorial(k-1)*np.math.factorial(n-k-1))
    
    for i,j in itertools.product(range(0,n*n-1), repeat=2):
        
        t = np.matmul(s,np.matmul(mat[i],mat[j],dtype=object),dtype=object)
        
        t[t!=0] = np.array([sp.nsimplify(o) for o in t[t!=0]],dtype=object)
        
        u = sp.nsimplify(np.trace(t))
        
        if u == sp.nsimplify(rs*Delta([i],[j])):
            l.append(True)
        else:
            l.append(False)
            print(u,sp.nsimplify(rs*Delta([i],[j])))
    
    
    if all(l) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
       
    return l        

In [72]:
superTr(3,2,TF32)

'Todas las relaciones se cumplen'

In [73]:
superTr(4,2,TF42)

'Todas las relaciones se cumplen'

In [74]:
superTr(4,3,TF43)

'Todas las relaciones se cumplen'

In [75]:
superTr(5,2,TF52)

'Todas las relaciones se cumplen'

In [76]:
superTr(5,3,TF53)

'Todas las relaciones se cumplen'

In [77]:
superTr(5,4,TF54)

'Todas las relaciones se cumplen'

### Las Cargas susy:

$(Q_{i'}^\alpha)^A_B = k! \Delta^A_{i'+4} \delta^\alpha_{B} \quad (\bar{Q}^{i'}_\alpha)^A_B = k!  \delta_\alpha^{A}\Delta_B^{i'+4}$

In [32]:
def char(d,k,a,i):
    
   # d = TraDelta(n,k)
    
    u = np.zeros((d+4,d+4),dtype=object)
    
    u[i+4][a] = np.math.factorial(k)*1
        
    return u
    
def charB(d,k,a,i):
    
   # d = TraDelta(n,k)
    
    u = np.zeros((d+4,d+4),dtype=object)
    
    u[a][i+4] = np.math.factorial(k)*1
        
    return u    

### Vamos a generarlas todas ahora

In [33]:
def Charges(n,k):
    
    d = TraDelta(n,k)
    p = {}
    
    for a in range(0,4):
        for i in range(0,d):
            
            p[(a,i)] = char(d,k,a,i)
            
    return p 

def ChargesB(n,k):
    
    d = TraDelta(n,k)
    p = {}
    
    for a in range(0,4):
        for i in range(0,d):
            
            p[(i,a)] = charB(d,k,a,i)
            
    return p    

In [34]:
Q32 = Charges(3,2)
QB32 = ChargesB(3,2)

In [35]:
Q42 = Charges(4,2)
QB42 = ChargesB(4,2)

Q43 = Charges(4,3)
QB43 = ChargesB(4,3)

In [36]:
Q52 = Charges(5,2)
QB52 = ChargesB(5,2)

Q53 = Charges(5,3)
QB53 = ChargesB(5,3)

Q54 = Charges(5,4)
QB54 = ChargesB(5,4)

### Los conmutadores 

$[T_I,Q^\alpha_{i'}] = i(t_I)_{i'}^{\ \ j'}Q^\alpha_{j'}$

$[T_I,\bar{Q}_\alpha^{i'}] = -i\bar{Q}_\alpha^{j'}(t_I)^{\ \ i'}_{j'}$

In [37]:
def rhsQ(n,k,Mat,tmat,I,i,a,st):
    
    d = TraDelta(n,k)
    
    p = np.zeros((d+4,d+4),dtype=object)
    
    for s in range(0,d):
        
        if st == 'Q':
            u = (tmat[I])[i][s]*Mat[(a,s)]
            u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
            p = p + u
            
        elif st == 'QB':
            
            u = (tmat[I])[s][i]*Mat[(s,a)]
            u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
            p = p + u
            
    if st == 'Q':
        return (1j)*p
        
    elif st == 'QB':
        return -(1j)*p

def algQ(n,k,Tmat,tmat,Mat,I,i,al,st):
    
    a = rhsQ(n,k,Mat,tmat,I,i,al,st)
    
    if st == 'Q':
        
        b = bra(Tmat[I],Mat[(al,i)])
    
    elif st == 'QB':
        
        b = bra(Tmat[I],Mat[(i,al)])
        
    ## Verifiquemos que todo sea del mismo tipo
    
    a[a!=0] = np.array([sp.nsimplify(s) for s in a[a!=0]],dtype=object)
    b[b!=0] = np.array([sp.nsimplify(s) for s in b[b!=0]],dtype=object)
    
    if (b == a).all():
        return True
    else:
        return False 
    
def AlgtestQ(n,k,Tmat,tmat,Mat,st):
    
    a = []
    
    d = TraDelta(n,k)
    
    for I in range(0,n*n-1):
        for al in range(0,4):
            for i in range(0,d):
                a.append(algQ(n,k,Tmat,tmat,Mat,I,i,al,st))
    
        # Para saber en que indices no se cumple
        
        #if alg2(n,i,j,Mat,estc,st) == False:
            
        #    print(i,j)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'    

In [191]:
AlgtestQ(3,2,TF32,T32,Q32,'Q')

'Todas las relaciones se cumplen'

In [192]:
AlgtestQ(3,2,TF32,T32,QB32,'QB')

'Todas las relaciones se cumplen'

In [193]:
AlgtestQ(4,2,TF42,T42,Q42,'Q')

'Todas las relaciones se cumplen'

In [198]:
AlgtestQ(4,2,TF42,T42,QB42,'QB')

'Todas las relaciones se cumplen'

In [201]:
AlgtestQ(4,3,TF43,T43,Q43,'Q')

'Todas las relaciones se cumplen'

In [203]:
AlgtestQ(4,3,TF43,T43,QB43,'QB')

'Todas las relaciones se cumplen'

In [204]:
AlgtestQ(5,2,TF52,T52,Q52,'Q')

'Todas las relaciones se cumplen'

In [205]:
AlgtestQ(5,2,TF52,T52,QB52,'QB')

'Todas las relaciones se cumplen'

In [206]:
AlgtestQ(5,3,TF53,T53,Q53,'Q')

'Todas las relaciones se cumplen'

In [207]:
AlgtestQ(5,3,TF53,T53,QB53,'QB')

'Todas las relaciones se cumplen'

In [208]:
AlgtestQ(5,4,TF54,T54,Q54,'Q')

'Todas las relaciones se cumplen'

In [209]:
AlgtestQ(5,4,TF54,T54,QB54,'QB')

'Todas las relaciones se cumplen'

### Supertraza de las cargas

In [38]:
def superTrQ(n,k,mat1,mat2):
    
    d = TraDelta(n,k)
    
    s = Gamma(n,k)
    
    l = []
    
#    rs = sp.Rational(np.math.factorial(n-2),2*np.math.factorial(k-1)*np.math.factorial(n-k-1))
    
    for i,j in itertools.product(range(0,d), repeat=2):
        for al,bl in itertools.product(range(0,4), repeat=2):
        
            t = np.matmul(s,np.matmul(mat1[(al,i)],mat2[(j,bl)],dtype=object),dtype=object)
        
            t[t!=0] = np.array([sp.nsimplify(o) for o in t[t!=0]],dtype=object)
        
            u = sp.nsimplify(np.trace(t))
        
            if u == sp.nsimplify(-np.math.factorial(k)**2*Delta([al],[bl])*Delta([i],[j])):
                l.append(True)
            else:
                l.append(False)
                print(u,sp.nsimplify(-Delta([al],[bl])*Delta([i],[j])))
    
    if all(l) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
       
    return l    

In [39]:
superTrQ(3,2,Q32,QB32)

'Todas las relaciones se cumplen'

In [40]:
superTrQ(4,2,Q42,QB42)

'Todas las relaciones se cumplen'

In [41]:
superTrQ(4,3,Q43,QB43)

'Todas las relaciones se cumplen'

In [43]:
superTrQ(5,2,Q52,QB52)

'Todas las relaciones se cumplen'

In [44]:
superTrQ(5,3,Q53,QB53)

'Todas las relaciones se cumplen'

In [45]:
superTrQ(5,4,Q54,QB54)

'Todas las relaciones se cumplen'

### Carga Central

$Z^A_B = iz(\delta^A_\alpha\delta_B^\alpha + 4 \Delta^A_{i'+4}\Delta^{i'+4}_B/d_n)$

In [39]:
def Z(n,k,c):
    
    d = TraDelta(n,k)
    
    u = np.zeros((d+4,d+4),dtype=object)
    
    for i in range(0,4):
        
        u[i,i] = (1j)*c
    
    for j in range(4,d+4):
        
        u[j,j] = (1j)*sp.Rational(4,d)*c
        
    return u     

### Relaciones de conmutacion

$[Z,Q^\alpha_{i'}] = iz(4/d_{n}-1)Q^\alpha_{i'}$

$[Z,\bar{Q}_\alpha^{i'}] = iz(4/d_{n}-1)\bar{Q}_\alpha^{i'}$

In [42]:
d = TraDelta(4,3)
d

4